In [3]:
import torch 
import torch.nn.functional as F
import math
import torch.nn as nn

## Scaled dot product attention
    - embed_size就是d_k， Q,K,V都是 batch, n_, dimension 这样的组成的话，embed_size就是Q.size(-1), 也就是dimension, 即 d_k
    - 这里mask，是把mask为0的地方全部取负无穷， -inf;softmax之后这些位置会变成0
    - softmax的dim = -1，意味着取最后一维做softmax

In [2]:
def scaled_dot_product_attention(Q,K,V,mask=None):
    
    # scale的sqrt dk, d_k = d_q
    embed_size = Q.size(-1)

    score = torch.matmul(Q, K.transpose(-2,-1)) / math.sqrt(embed_size)

    if mask is not None:
        score = score.masked_fill(mask==0, float('-inf'))
    attention = F.softmax(score,dim = -1)

    output = torch.matmul(attention, V)

    return attention, output

## 单头注意力机制，这里有输入的 线性变换
- nn.Linear(in_features,out_features)是线性变化层，定义了一个输入维度in_feature,输出维度out_feature的线性变换层
- 这里的线性变换是 
$$y = xW^t + b $$

- 也就是 这里单头注意力机制，就是将输入的序列，首先进行线性变换成查询矩阵，key矩阵，value矩阵，然后进行scaled dot-product attention

In [ ]:
class Attention(nn.Module):
    def __init__(self, embed_size):
        """
        单头注意力机制

        参数embed_size, 输入序列的嵌入维度
        """
        super(Attention, self).__init__()

        self.embed_size = embed_size
        # 定义线性层，用于生成查询、键和值矩阵
        self.w_q = nn.Linear(embed_size, embed_size)
        self.w_k = nn.Linear(embed_size, embed_size)
        self.w_v = nn.Linear(embed_size, embed_size)

    def forward(self, q, k, v, mask = None):
        """
        前向传播函数

        参数：
        q:查询矩阵， batch_size, seq_len_q, embed_size
        k: 键矩阵，batch_size, seq_len_k, embed_size
        v: 值矩阵，batch_size, seq_len_v, embed_size

        return:
            output:注意力加权之后的输出
            atten:注意力权重

        """
        Q = self.w_q(q)
        K = self.w_k(k)
        V  = self.w_v(v)

        attention,out = scaled_dot_product_attention(Q,K,V, mask)
        return attention,out


## 掩码机制
- 当需要掩藏未来的词，就需要mask
- 发生在softmax计算之前
    - 首先，$$QK^T$$,这里的计算还是需要所有的值
    - 在计算score的时候，所有位置保留
    - 在计算softmax之前排除，这样就可以让模型看不到未来词，来进行下一个词的预测
- 这里是look ahead mask， 用于decoder

- 如果是padding mask,是处理填充的符，在encoder,decoder里面都要mask

![mask](./images/mask.png)

## 自注意力机制，交叉注意力机制
- 自注意力机制的Q, K, V同源
- transformer的decoder交叉注意力机制，Q 来自上一个decoder block， K,V来自encoder的输出

In [ ]:
class SelfAttention(nn.Module):

    def __init__(self, embed_size):

        """
        自注意力（Self-Attention）机制。
        
        参数:
            embed_size: 输入序列的嵌入维度（每个向量的特征维度）。
        """
        super(SelfAttention, self).__init__()
        self.attention = Attention(embed_size)
    
    def forward(self,x, mask=None):

        """
        前向传播函数。
        
        参数:
            x: 输入序列 (batch_size, seq_len, embed_size)
            mask: 掩码矩阵 (batch_size, seq_len, seq_len)

        返回:
            out: 自注意力加权后的输出 (batch_size, seq_len, embed_size)
            attention_weights: 注意力权重矩阵 (batch_size, seq_len, seq_len)
        """

        out, attention = self.attention(x,x,x,mask)
        return out, attention

## 交叉注意力机制

$$Q = X_{decoder} W^Q $$
$$ K = X_{encoder} W^K $$
$$V = X_{encoder} W^V $$

In [ ]:
class CrossAttention(nn.Module):
    def __init__(self, embed_size):
        super(CrossAttention,self).__init__()
        self.attention = SelfAttention(embed_size)

    def forward(self, q, kv, mask=None):
        """
        前向传播函数。
        
        参数:
            query: 查询矩阵的输入 (batch_size, seq_len_q, embed_size)
            kv: 键和值矩阵的输入 (batch_size, seq_len_kv, embed_size)
            mask: 掩码矩阵 (batch_size, seq_len_q, seq_len_kv)

        返回:
            out: 注意力加权后的输出 (batch_size, seq_len_q, embed_size)
            attention_weights: 注意力权重矩阵 (batch_size, seq_len_kv, seq_len_kv)
        """
        # 在交叉注意力机制中，q 和 k, v 不同
        # q 来自解码器，k 和 v 来自编码器（观察模型架构图）
        out,atten = self.attention(q,kv,kv,mask = mask)

        return out,atten
    
    

## 多头注意力机制
- 用多个注意力头，并行地关注输入数据在不同维度上的依赖关系
- 多个头，每个头有独立的线性变化，
    - $head_i =  Attention(QW^Q_{i},KW^K_{i},VW^V_{i})$
    - 最后沿最后一维拼接，通过线性变换矩阵$W^Q$映射会原始嵌入维度embed_size
    -  $MultiHead(Q,K,V) = Concat(head_1,...m head_h)W^o$
- 映射回原始维度的主要目的是为了实现残差连接（Residual Connection）,张量维度需要匹配


###
nn.ModuleList([nn.Linear(embed_size, embed_size) for _ in range(num_heads)])

In [ ]:
# 一个凭借直觉的写法

class MultiHeadAttention(nn.Module):
    def __init__(self, embed_size, num_heads):
        """
        多头注意力机制
        参数：
        embed size
        num_heads

        """
        super(MultiHeadAttention,self).__init__()
        self.embed_size = embed_size
        self.num_heads = num_heads

        self.w_q = nn.ModuleList([nn.Linear(embed_size, embed_size) for _ in range(num_heads)])
        self.w_k = nn.ModuleList([nn.Linear(embed_size, embed_size) for _ in range(num_heads)])
        self.w_v = nn.ModuleList([nn.Linear(embed_size, embed_size) for _ in range(num_heads)])

        self.fc_out = nn.Linear(num_heads*embed_size, embed_size)
    
    def forward(self, q,k,v,mask = None):
        batch_size = q.size(0)
        multi_head_outputs = []

        for i in range(self.num_heads):
            Q = self.w_q[i](q)
            K = self.w_k[i](k)
            V = self.w_v[i](v)

            attention, _ = scaled_dot_product_attention(Q,K,V,mask)
            multi_head_outputs.append(attention)
        
        concat_out = torch.cat(multi_head_outputs,dim=-1)
        # 把多个张量沿某个维度拼接起来, 这里就是沿最后一个维度，embed_size拼接，所以最后拼接了num_head次
        out = self.fc_out(concat_out)
        return out
    

    # 每个头都用独立的线性变换，且维度是embed_size,模型的参数量非常大，性能的提升会不会和参数量变大有关
    # 如果想评估性能提升的关键之处
    # 1.把单头注意力的参数量提高到多头的量，也就是说 原本维度是d*d,现在维度是d*(d*h)
    # 2.把每个多头的维度都降低，到跟单头一样，也就是每个线性层是 d*(d/num_head)

In [ ]:
# 将多头的每个头维度控制为 embed_size/num_head

class MultiHeadAttention(nn.Module):
    def __init__(self, embed_size, num_heads):
        super(MultiHeadAttention, self).__init__()
        assert embed_size % num_heads == 0, "embed_size 必须能被 num_heads 整除。"
        #强制检查这个条件是否符合，否则报出错误
        self.embed_size = embed_size
        self.num_heads = num_heads
        self.head_dim = embed_size // num_heads  # 每个头的维度

        # nn.linear(输入features，输出的features)
        self.w_q = nn.ModuleList([nn.Linear(embed_size, self.head_dim) for _ in range(num_heads)])
        self.w_k = nn.ModuleList([nn.Linear(embed_size, self.head_dim) for _ in range(num_heads)])
        self.w_v = nn.ModuleList([nn.Linear(embed_size, self.head_dim) for _ in range(num_heads)])

        # 输出线性层，将多头拼接后的输出映射回 embed_size
        self.fc_out = nn.Linear(embed_size, embed_size)

    def forward(self, q, k, v, mask=None):
        """
        前向传播函数。
        
        参数:
            q: 查询矩阵 (batch_size, seq_len_q, embed_size)
            k: 键矩阵 (batch_size, seq_len_k, embed_size)
            v: 值矩阵 (batch_size, seq_len_v, embed_size)
            mask: 掩码矩阵 (batch_size, seq_len_q, seq_len_k)

        返回:
            out: 注意力加权后的输出
            attention_weights: 注意力权重矩阵
        """
        batch_size = q.shape[0]
        multi_head_outputs = []

        # 针对每个头独立计算 Q, K, V，并执行缩放点积注意力
        for i in range(self.num_heads):
            Q = self.w_q[i](q)  # (batch_size, seq_len_q, head_dim)
            K = self.w_k[i](k)  # (batch_size, seq_len_k, head_dim)
            V = self.w_v[i](v)  # (batch_size, seq_len_v, head_dim)

            # 执行缩放点积注意力
            scaled_attention, _ = scaled_dot_product_attention(Q, K, V, mask)
            multi_head_outputs.append(scaled_attention)

        # 将所有头的输出拼接起来
        concat_out = torch.cat(multi_head_outputs, dim=-1)  # (batch_size, seq_len_q, embed_size)

        # 通过输出线性层
        out = self.fc_out(concat_out)  # (batch_size, seq_len_q, embed_size)

        return out
    
# 这个代码使用了for循环逐一计算每个头的线性变换，但是，不是很计算高效


## 多头的实现，优雅实现
- 原本是为每个头单独创建线性层，现在创造一个共享线性层
    ```
    ### “共享”的 Q, K, V 线性层
    self.w_q = nn.Linear(embed_size, embed_size)
    self.w_k = nn.Linear(embed_size, embed_size)
    self.w_v = nn.Linear(embed_size, embed_size)
    ```
- 原本是循环遍历每个层，计算score
    - 一次性计算Q，K,V，用reshape, transpose，将矩阵拆分为多头的格式
    - 如上，self.w_q（q）是batch, n_q, embed_size
        - `Q.reshape(batch, n_q, num_heads, head_dim).transpose(1,2)`
        - 拆分了embed_size，之后，将形状转变为batch, num_heads, n_q, head_dim
        - 这样做的目的是，后续的每个头的计算是独立的
        - 如果不用reshape,用view的话，需要在拼接的时候先用contiguous()，因为view要求输入张量在内存上连续

### 原本的scaled-dot-product attention也需要修改，因为 形状的改变
```
def scaled_dot_product_attention(Q, K, V, mask=None):
	"""
    缩放点积注意力计算。
    参数:
        Q: 查询矩阵 (batch_size, num_heads, seq_len_q, head_dim)
        K: 键矩阵 (batch_size, num_heads, seq_len_k, head_dim)
        V: 值矩阵 (batch_size, num_heads, seq_len_v, head_dim)
        mask: 掩码矩阵 (1, 1, seq_len_q, seq_len_k) 或 (batch_size, 1, seq_len_q, seq_len_k) 或 (batch_size, num_heads, seq_len_q, seq_len_k)

    返回:
        output: 注意力加权后的输出矩阵
        attention_weights: 注意力权重矩阵
    """
    ...（操作依旧不变，只需要改注释）
    return output, attention_weights    
```

### 广播机制broadcasting
- 用一个较小形状的张量与较大的张量做运算时，只要维度是兼容的，就会自动复制扩展来匹配
- 所以 (1, 1, seq_len_q, seq_len_k) 可以自动广播到 (batch_size, num_heads, seq_len_q, seq_len_k)。
- 维度广播，因为维度兼容，自动复制扩展到大的维度形状